In [1]:
import numpy as np
import pandas as pd

Gene map

In [2]:
big_block=pd.read_csv(r'big_clock_key_gene.csv')

In [3]:
big_block_INFOR=pd.read_table(r'Human.GRCh38.p13.annot.tsv',sep='\t')

C:\Users\11340\AppData\Local\Temp\ipykernel_17680\1890634434.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  big_block_INFOR=pd.read_table(r'D:\metabolic network\转录组\Wgcna\Human.GRCh38.p13.annot.tsv',sep='\t')


pathway information

In [9]:
big_block_key=pd.merge(big_block,big_block_INFOR,left_on='gene',right_on='GeneID')
pathway_df = pd.read_table(r'NASH_ontology_network.txt', sep='\t')
pathway_df_KEGG = pathway_df[pathway_df['DB'] == 'KEGG']
KEY_PATHWAY = pd.read_csv(r'KEY_PATHWAY.csv')
KEY_gene_PATHWAY = pathway_df_KEGG[pathway_df_KEGG['term_id'].isin(list(KEY_PATHWAY['Term']))]
sim_total = []
for i in big_block_key['EnsemblGeneID']:

    pathway_num = pathway_df_KEGG[pathway_df_KEGG['gene_id'] == i]
    if pathway_num.shape[0] == 0:
        sim = 0
    else:
        sim = KEY_gene_PATHWAY[KEY_gene_PATHWAY['gene_id'] == i].shape[0] / pathway_num.shape[0]

    sim_total.append(sim)
big_block_key['sim']=sim_total

computation of network information(traditional method,GAPR,finder,pathway))

In [32]:
import pandas as pd
import networkx as nx
import os

# Load the protein-protein interaction network data
block_ppi_path = r'block_ppi_GAPR.txt'

# 检查文件是否存在
if not os.path.exists(block_ppi_path):
    raise FileNotFoundError(f"The file {block_ppi_path} does not exist.")

block_ppi_data = pd.read_csv(block_ppi_path, sep='\t')

# Create a graph from the edge list
block_G = nx.from_pandas_edgelist(block_ppi_data, '#Protein A', 'Protein B')

# Calculate k-shell
NASH_k_shell = nx.core_number(block_G)

# Calculate degree
NASH_degree = dict(block_G.degree())

# Calculate Pagerank
NASH_pagerank = nx.pagerank(block_G)

# Calculate betweenness centrality
NASH_betweenness = nx.betweenness_centrality(block_G)

# 如果没有 nx.collective_influence 方法，可以注释掉或实现自己的计算
#NASH_collective_influence = nx.collective_influence(block_G)

# 构建 DataFrame
NASH_nodes = list(block_G.nodes())
NASH_scores_df = pd.DataFrame({
    'entry_id': NASH_nodes,
    'K-Shell': [NASH_k_shell[node] for node in NASH_nodes],
    'Degree': [NASH_degree[node] for node in NASH_nodes],
    'PageRank': [NASH_pagerank[node] for node in NASH_nodes],
    'Betweenness': [NASH_betweenness[node] for node in NASH_nodes]
    # 如果实现了集体影响力计算，可以取消注释以下一行
    # ,'Collective_influence': [NASH_collective_influence[node] for node in NASH_nodes]
})

# 打印前几行以验证结果
print(NASH_scores_df.head())
def calculate_collective_influence(G, l=2):
    CI = {}
    for node in G.nodes():
        # 集体影响力的计算公式可以是 CI(node) = (k - 1) * Σ邻居(k_neigh - 1)
        neighbors = list(G.neighbors(node))
        influence_sum = sum([G.degree(neigh) - 1 for neigh in neighbors])
        CI[node] = (G.degree(node) - 1) * influence_sum
    return CI

# 使用函数计算集体影响力
NASH_collective_influence = calculate_collective_influence(block_G)

# 然后将其添加到 DataFrame 中
NASH_scores_df['Collective_influence'] = [NASH_collective_influence[node] for node in NASH_nodes]

   entry_id  K-Shell  Degree  PageRank  Betweenness
0         2       10      26  0.000921     0.002458
1       351       15     340  0.013429     0.134389
2       427        8      10  0.000425     0.000987
3       476       13      19  0.000571     0.000450
4       506       14      32  0.001018     0.004709


In [148]:
big_block_key_all=pd.merge(big_block_key,NASH_scores_df,left_on='gene',right_on='entry_id')

In [38]:
big_block_key_all

,0,gene,renum,From,To,Species,Gene Name,Score,Gene Symbol,Description_x,...,GOFunction,GOProcess,GOComponent,sim,entry_id,K-Shell,Degree,PageRank,Betweenness,Collective_influence
0,161,351,161,APP,351,Homo sapiens,amyloid beta precursor protein(APP),9.919679e-01,APP,Amyloid Beta Precursor Protein,...,RNA polymerase II cis-regulatory region sequen...,G2/M transition of mitotic cell cycle///microg...,extracellular region///extracellular space///n...,0.000,351,15,340,0.013429,0.134389,1681101
1,1243,3172,1243,HNF4A,3172,Homo sapiens,hepatocyte nuclear factor 4 alpha(HNF4A),9.688534e-01,HNF4A,Hepatocyte Nuclear Factor 4 Alpha,...,transcription cis-regulatory region binding///...,type B pancreatic cell development///regulatio...,chromatin///nucleus///nucleoplasm///cytoplasm,0.000,3172,15,404,0.015351,0.148791,2202395
2,2143,6872,2143,TAF1,6872,Homo sapiens,TATA-box binding protein associated factor 1(T...,9.405395e-01,TAF1,TATA-Box Binding Protein Associated Factor 1,...,RNA polymerase II core promoter sequence-speci...,negative regulation of transcription by RNA po...,chromatin///nucleus///nucleoplasm///transcript...,0.000,6872,15,382,0.013531,0.127781,2215134
3,1419,4149,1419,MAX,4149,Homo sapiens,MYC associated factor X(MAX),9.164938e-01,MAX,MYC Associated Factor X,...,RNA polymerase II transcription regulatory reg...,negative regulation of transcription by RNA po...,chromatin///nucleus///nucleoplasm///cytoplasm/...,0.000,4149,15,163,0.005253,0.022825,566838
4,118,8452,118,CUL3,8452,Homo sapiens,cullin 3(CUL3),9.126446e-01,CUL3,Cullin 3,...,ubiquitin-protein transferase activity///Notch...,G1/S transition of mitotic cell cycle///negati...,spindle pole///nucleus///nucleoplasm///cytopla...,0.000,8452,15,188,0.005816,0.035568,843744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,1305,3586,1305,IL10,3586,Homo sapiens,interleukin 10(IL10),2.590000e-06,IL10,Interleukin 10,...,cytokine activity///interleukin-10 receptor bi...,negative regulation of cytokine production///p...,extracellular region///extracellular space,0.175,3586,3,4,0.000196,0.000060,306
273,172,8567,172,MADD,8567,Homo sapiens,MAP kinase activating death domain(MADD),9.700000e-07,MADD,MAP Kinase Activating Death Domain,...,guanyl-nucleotide exchange factor activity///d...,cell surface receptor signaling pathway///acti...,cytoplasm///cytosol///plasma membrane///membra...,0.000,8567,6,8,0.000364,0.001216,4683
274,1665,5607,1665,MAP2K5,5607,Homo sapiens,mitogen-activated protein kinase kinase 5(MAP2K5),6.500000e-07,MAP2K5,Mitogen-Activated Protein Kinase Kinase 5,...,protein kinase activity///protein serine/threo...,negative regulation of transcription by RNA po...,nucleus///spindle///cytosol,0.000,5607,6,8,0.000306,0.000244,6426
275,1863,6351,1863,CCL4,6351,Homo sapiens,C-C motif chemokine ligand 4(CCL4),6.500000e-07,CCL4,C-C Motif Chemokine Ligand 4,...,cytokine activity///protein binding///chemokin...,monocyte chemotaxis///inflammatory response///...,extracellular region///extracellular space,0.025,6351,4,5,0.000275,0.000549,1616


In [150]:
GAPR_results=pd.read_excel(r'test_BLOCK_GAPR_AP.xlsx')
big_block_key_all=pd.merge(big_block_key_all,GAPR_results,left_on='gene',right_on='AP')

Comstruction of the final score

In [73]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
big_block_key_all['Score_norm'] = scaler.fit_transform(big_block_key_all[['Score']])
big_block_key_all['K-Shell_norm'] = scaler.fit_transform(big_block_key_all[['K-Shell']])
big_block_key_all['Degree_norm'] = scaler.fit_transform(big_block_key_all[['Degree']])
big_block_key_all['PageRank_norm'] = scaler.fit_transform(big_block_key_all[['PageRank']])
big_block_key_all['Betweenness_norm'] = scaler.fit_transform(big_block_key_all[['Betweenness']])
big_block_key_all['Collective_influence_norm'] = scaler.fit_transform(np.log10(1+big_block_key_all[['Collective_influence']]))
big_block_key_all['LCC changes_norm'] = scaler.fit_transform(big_block_key_all[['LCC changes']])
big_block_key_all['Score_total'] = big_block_key_all['Score_norm']*(1/-np.log2(big_block_key_all['LCC changes']+0.0001))*big_block_key_all['sim']
big_block_key_all.sort_values(by='Score_total', ascending=False).to_csv(r'big_block_key_all_final.csv', index=False)
big_block_key_all['Score_total'] = big_block_key_all['Score'] * big_block_key_all['LCC changes'] * big_block_key_all[
    'sim']
th_17_path = pd.read_csv(r'TH17_pathway.csv', header=None)
big_block_key_all[big_block_key_all['From'].isin(th_17_path[0])]